In [1]:
import pandas as pd
import numpy as np
import json
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import precision_score, recall_score, accuracy_score, mean_squared_error

# For each ticker (company), store all news into a dataframe

In [2]:
# Return one season's financial news dataframe
def season_dataframe(ticker, season):
    with open('./financial_news/'+ticker+str(season), 'r') as ticker_news:
        content  = ticker_news.read()
    records = json.loads(content)
    df = pd.DataFrame(records)
    return df

In [3]:
# Create dataframe for each ticker, storing its news
#ALL_TICKERS = ["$TSLA", "$NVDA", "$INTC", "$PFE", "$SPGI", "$LRCX", "$TMUS", "$ADSK", "$VRTX", "$TWTR", "$EBAY", "$CARR", "$VRSN", "$GRMN", "$ANET", "$AAL"]
ALL_TICKERS = ["$TSLA", "$NVDA", "$INTC", "$PFE", "$SPGI", "$LRCX", "$TMUS", "$ADSK", "$VRTX", "$TWTR", "$EBAY", "$CARR", "$GRMN", "$ANET", "$AAL"]
#ALL_TICKERS = ["$TSLA"]
df = []
for ticker in ALL_TICKERS:
  # Concatenate 8 seasons into one dataframe
  df_temp = pd.DataFrame()
  for i in range(1, 9):
      new_df = season_dataframe(ticker, i)
      df_temp = pd.concat([df_temp, new_df], ignore_index=True)
  df.append(df_temp)

Usage: df[0] will return $TSLA news from 2019-01-01 to 2020-12-31

# Calculate Polarity of each news, using nltk.sentiment.vader package

In [4]:
# Calculate vader
vader = SentimentIntensityAnalyzer()

# Helper function, which calculates the sentiment and returns compund score
def cal_compound(t):
    return vader.polarity_scores(t)["compound"]

In [6]:
for each_df in df:
  each_df['title_compound'] = each_df['title'].apply(cal_compound)
  each_df['text_compound'] = each_df['text'].apply(cal_compound)

Convert publishedDate to YYYY-MM-DD

In [7]:
# Helper function: for the date conversion
def remove_time(publish_date):
  return publish_date[0:10]

In [8]:
for each_df in df:
  each_df['publishedDate'] = each_df['publishedDate'].apply(remove_time)

Convert weekend to next Monday

In [9]:
# Helper function: Convert weekend to next Monday
def moveWeekend(publish_date):
  d = datetime.datetime(int(publish_date[0:4]), int(publish_date[5:7]), int(publish_date[8:10]))
  if (d.weekday() == 5):
    return str(d+datetime.timedelta(days=2))[0:10]
  elif (d.weekday() == 6):
    return str(d+datetime.timedelta(days=1))[0:10]
  else:
    return publish_date

In [10]:
for each_df in df:
  each_df['publishedDate'] = each_df['publishedDate'].apply(moveWeekend)

# Calculate title and text's daily polarity mean respectively

In [11]:
polarity = []
# Calcuate mean on daily basis
for each_df in df:
  each_title_mean = each_df.groupby('publishedDate', as_index=False)['title_compound'].mean()
  each_text_mean = each_df.groupby('publishedDate', as_index=False)['text_compound'].mean()
  each_polarity = pd.merge(each_title_mean, each_text_mean, on='publishedDate')
  each_polarity['symbol'] = each_df['symbol']
  polarity.append(each_polarity)

# Merge everyday's closing price with polarity dataframe

In [12]:
# Helper function: Fetch closing price by a specific date and ticker name
def fetchClosingPrice(time, ticker):
  data = pd.read_csv("./stock_price/compare_previous_day/"+ticker+".csv")
  data = data[data.Date.isin([time])]
  data = data['Close']
  #return data.iloc[0].item()
  return 0 if len(data.index) == 0 else data.iloc[0].item()

In [13]:
for each_polarity in polarity:
  each_polarity['close_price'] = each_polarity['publishedDate'].apply(fetchClosingPrice, args=(each_polarity.symbol.iloc[0],))

# Merge everyday's indicator with polarity dataframe

In [14]:
def fetchIndicator(time, ticker):
  data = pd.read_csv("./stock_price/compare_previous_day/"+ticker+".csv")
  data = data[data.Date.isin([time])]
  data = data['indicator']
  #return data.iloc[0].item()
  return 0 if len(data.index) == 0 else data.iloc[0].item()

In [15]:
for each_polarity in polarity:
  each_polarity['indicator'] = each_polarity['publishedDate'].apply(fetchIndicator, args=(each_polarity.symbol.iloc[0],))

In [16]:
polarity[0]

,publishedDate,title_compound,text_compound,symbol,close_price,indicator
0,2019-01-01,-0.226300,-0.296000,TSLA,0.000000,0.0
1,2019-01-03,-0.526700,-0.128000,TSLA,60.071999,0.0
2,2019-01-07,0.000000,0.000000,TSLA,66.991997,1.0
3,2019-01-17,-0.273200,0.025800,TSLA,69.461998,1.0
4,2019-01-18,-0.340000,-0.440400,TSLA,60.452000,0.0
...,...,...,...,...,...,...
429,2020-12-25,0.148000,0.509500,TSLA,0.000000,0.0
430,2020-12-28,0.081800,0.378638,TSLA,663.690002,1.0
431,2020-12-29,0.209004,0.202939,TSLA,665.989990,1.0
432,2020-12-30,0.209600,0.323291,TSLA,694.780029,1.0


# Add S&P 500 Adj Close into dataframe

In [18]:
sp_df = pd.read_csv("./stock_price/original/S&P500.csv")
sp_df

,Date,Open,High,Low,Close
0,12-31-2020,"3,733.27","3,760.20","3,726.88",3756.07
1,12-30-2020,"3,736.19","3,744.63","3,730.21",3732.04
2,12-29-2020,"3,750.01","3,756.12","3,723.31",3727.04
3,12-28-2020,"3,723.03","3,740.51","3,723.03",3735.36
4,12-24-2020,"3,694.03","3,703.82","3,689.32",3703.06
...,...,...,...,...,...
500,01-08-2019,"2,568.11","2,579.82","2,547.56",2574.41
501,01-07-2019,"2,535.61","2,566.16","2,524.56",2549.69
502,01-04-2019,"2,474.33","2,538.07","2,474.33",2531.94
503,01-03-2019,"2,491.92","2,493.14","2,443.96",2447.89


In [19]:
def reorder_date(date):
  return date[6:10]+'-'+date[0:5]

In [20]:
sp_df['publishedDate'] = sp_df['Date'].apply(reorder_date)
sp_df = sp_df[['publishedDate', 'Close']]

In [21]:
sp_df = sp_df.iloc[::-1]

In [22]:
sp_df = sp_df.rename(columns= {'Close': "s&p500_close_price"}, inplace=False)

In [23]:
# Convert str to float
sp_df['s&p500_close_price'] = pd.to_numeric(sp_df['s&p500_close_price'], downcast="float")

In [24]:
for i in range(len(polarity)):
  polarity[i] = pd.merge(polarity[i], sp_df, on="publishedDate")


# Training and Testing Step

In [ ]:
for each_polarity in polarity:
  y = each_polarity['indicator']
  X = each_polarity.drop(columns=['indicator','publishedDate','symbol'])
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)
  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)
  
  # apply RandomForestRegressor
  out_date = each_polarity.publishedDate.values
  rf = RandomForestRegressor(n_estimators=150)
  rf.fit(X_train, y_train)
  y_pred = rf.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  corr = np.corrcoef(X_train, y_train, rowvar=False)[-1, :-1]
  corr = corr.max()
  f = open(each_polarity.symbol.iloc[0]+".summary.csv", "w")
  f.write("MSE,correlation\n")
  f.write("{},{:.2f}\n".format(mse, corr))
  f.close()

  f = open(each_polarity.symbol.iloc[0]+".output.csv", "w")
  f.write("date,predicted_indicator\n")
  for i in range(y_pred.shape[0]):
      f.write("{},{}\n".format(out_date[i], y_pred[i]))
  f.close()

In [25]:
from sklearn.neural_network import MLPClassifier

In [54]:
avg_accuracy = 0
for each_polarity in polarity:
  y = each_polarity['indicator']
  X = each_polarity.drop(columns=['indicator','publishedDate','symbol'])
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)
  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)
  
  # apply MLPClassifier
  out_date = each_polarity.publishedDate.values
  #rf = RandomForestRegressor(n_estimators=150)
  nn = MLPClassifier(
    hidden_layer_sizes=(90,10),
    random_state=0,
    max_iter=60,
  )
  nn.fit(X_train, y_train)
  #rf.fit(X_train, y_train)
  y_pred = nn.predict(X_test)
  #print(accuracy_score(y_test, y_pred))
  avg_accuracy+=accuracy_score(y_test, y_pred)
  #mse = mean_squared_error(y_test, y_pred)
  #corr = np.corrcoef(X_train, y_train, rowvar=False)[-1, :-1]
  #corr = corr.max()
  """ accuracy = accuracy_score(y_test, y_pred)
  f = open(each_polarity.symbol.iloc[0]+".summary.csv", "w")
  f.write("accuracy\n")
  f.write("{:.2f}\n".format(accuracy))
  f.close()

  f = open(each_polarity.symbol.iloc[0]+".output.csv", "w")
  f.write("date,predicted_indicator\n")
  for i in range(y_pred.shape[0]):
      f.write("{},{}\n".format(out_date[i], y_pred[i]))
  f.close() """
print(avg_accuracy/15)

/Users/vincent/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (60) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/vincent/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (60) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/vincent/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (60) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/vincent/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (60) reached and the optimi

0.5529588853897907


/Users/vincent/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (60) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
